# 1. 미분 자동 계산

## 1. 상자로서의 변수

* 첫 번째 단계에서는 DeZero의 구성 요소인 변수를 만듭니다

### 1. 변수란

* 상자에 데이터를 넣는 느낌 => 상자가 변수

### 2. Variable 클래스 구현

* 변수라는 개념을 Variable이라는 이름의 클래스로 구현

In [251]:
class Variable:
    def __init__(self, data):
        self.data = data

In [252]:
import numpy as np

data = np.array(1.0)
x = Variable(data) #x는 Variable의 인스턴스
print(x.data)

1.0


In [253]:
x.data = np.array(2.0)
print(x.data)

2.0


### 3. 넘파이 다차원 배열

* 다차원 배열에서 원소의 순서에는 방향이 있고 이 방향을 차원 혹은 축 이라고 한다

* 0차원 배열 : 스칼라
* 1차원 배열 : 벡터
* 2차원 배열 : 행렬

* 3차원 배열과 3차원 벡터는 다른 의미 => 3차원 배열 : [[[1]]], 3차원 벡터 :[1,2,3]

In [254]:
import numpy as np

x = np.array(1) # np.array(1) != np.array([1])
print(x.ndim)

x = np.array([1,2,3])
print(x.ndim)

x = np.array([[1,2,3],
              [4,5,6]])
print(x.ndim)

0
1
2


## 2. 변수를 낳는 함수

### 1. 함수란 

* 어떤 변수로부터 다른 변수로의 대응 관계를 정한  것
* 변수 사이의 대응 관계를 정하는 역할을 함수가 맡게 됨

### 2. Function 클래스 구현

* Function 클래스는 Variable 인스턴스를 입력받아 Variable 인스턴스를 출력
* Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 있다

In [255]:
class Function:
    def __call__(self, input): #call 메서드는 객체를 함수처럼 이용하게 만든다
        x = input.data # 데이터를 꺼낸다
        y = x**2 # 실제 계산 
        output = Variable(y) # Variable 형태로 되돌린다 
        return output

참고 : \_\_call__ 사용 이유

In [256]:
class TempFunction1:
    def calculate(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output
    
x = Variable(np.array(10))
t_f = TempFunction1()
try:
    t_y = t_f(x)
    print(t_y.data)
except:
    print('error') # __call__ 메서드를 사용하지 않으면 f_t(x)형태로 바로 사용하지 못함

error


In [257]:
t_f = TempFunction1()
t_y = t_f.calculate(x) # 이런식으로 직접 메서드를 호출해서 사용해야함
t_y.data

100

In [258]:
class TempFunction2:
    def __call__(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output
    
x = Variable(np.array(10))
t_f = TempFunction2()
try:
    t_y = t_f(x)
    print(t_y.data)
except:
    print('error') # __call__ 메서드를 이용하여 바로 f(...)형태로 불러올 수 있게 만듬

100


### 3. Function 클래스 이용

In [259]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y)) # 반환형태는 Variable클래스
print(y.data)

<class '__main__.Variable'>
100


> DeZero 클래스 규칙
* 지금 구현된 Function은 입력값의 제곱으로 고정된 함수
* Function 클래스는 기반 클래스로서 모든 함수에 ```공통되는``` 기능을 구현
* 구체적인 함수는 Function 클래스를 ```상속한 클래스에서 구현```


In [260]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 함수 호출시 자연스럽게 순전파 진행
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError() # 이 메서드는 상속을 통해 구현 필요

In [261]:
# forward class 자체를 사용하는 경우

try:
    x = Variable(np.array(10))
    t_f = Function()
    t_f(x) #forward가 정의되있지 않기 때문에 오류 발생
except NotImplementedError as e:
    print('error')

error


In [262]:
class Square(Function):
    def forward(self, x):
        return x**2
    
# Square 클래스는 Function 클래스를 상속하기 때문에 __call__메서드는 Square에서 사용 가능

In [263]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## 3. 함수 연결

* Square라는 제곱 계산용 함수 클래스이외에 다른 계산 클래스를 구현

### 1. Exp 함수 구현

In [264]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

### 2. 함수 연결

In [265]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## 4. 수치 미분

### 1. 미분이란

* 변화율
* $\displaystyle{f(x) = lim_{h \to 0} \frac{f(x+h) - f(x)}{h}}$

### 2. 수치 미분 구현

* 컴퓨터는 극한을 취급할수 없기때문에 $h=0.0001$과 같은 매우 작은 값을 이용하여 미분 계산
* 이러한 방법을 수치 미분이라함

* 수치미분의 오차를 중앙 차분을 통해 줄일수 있다
* $\displaystyle{f(x) = lim_{h \to 0} \frac{f(x+h) - f(x-h)}{2h}}$

In [266]:
def numerical_diff(f, x, eps = 1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    
    y0 = f(x0)
    y1 = f(x1)
    
    return (y1.data - y0.data) / (2 * eps)

In [267]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


### 3. 합성 함수의 미분

In [268]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


### 4. 수치 미분의 문제점

* 자릿수 누락때문에 오차가 포함되기 쉽다

## 5. 역전파 이론

* 역전파를 이용하면 미분을 효율적으로 계산할 수 있고 결과값의 오차도 더 작게 구할수 있다.

### 1. 연쇄법칙

$y = F(x)$라는 함수가 있을때  
$a = A(x), b = B(a), y = C(b)$로 세 함수로 구성되있음  
이때 $\displaystyle\frac{\partial{dy}}{\partial{dx}} = \frac{\partial{dy}}{\partial{db}}\frac{\partial{db}}{\partial{da}}\frac{\partial{da}}{\partial{dx}}$ 로 표현할수 있다  


### 2. 역전파 원리 도출

* 출력에서 입력방향으로 계산

### 3. 계산 그래프로 살펴보기

* 역전파는 순전파때의 결과값이 필요하다 (매우중요)

## 6. 수동 역전파

### 1. Variable 클래스 추가 구현

In [269]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None # 나중에 실제 gradient를 구한뒤 값을 할당

### 2. Function 클래스 추가 구현

In [270]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input #입력데이터 기억, 역전파에 사용됨
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()

### 3. Square와 Exp 클래스 추가 구현

In [271]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data 
        gx = 2 * x * gy #gy는 출력쪽에서 전해지는 미분값을 전달하는 역할
        return gx

In [272]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data # 상위 클래스인 Function에서 기억하고있는 input.data불러오기
        gx = np.exp(x) * gy
        return gx

### 4. 역전파 구현

* 역전파를 구현을 했지만 순서대로 하나하나 다시 역전파를 실행해줘서 직접 기울기를 집어 넣어야 되서 불편함

In [273]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [274]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad) # b.grad는 y에 대한 b의 변화량
a.grad = B.backward(b.grad) # a.grad는 y에 대한 a의 변화량
x.grad = A.backward(a.grad) # x.grad는 y에 대한 x의 변화량
print(x.grad)

3.297442541400256


In [275]:
2*np.exp(0.5)

3.2974425414002564

## 7. 역전파 자동화

* 순전파를 수행하고 한번에 역전파를 수행하는 방식으로 수정

### 1. 역전파 자동화 시작

In [491]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None # 변수가 어떤 함수로 부터 만들어 지는지 기록
        
    def set_creator(self, func):
        self.creator = func
        
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output #출력도 추가로 저장
        return output
        
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = gy * np.exp(x)
        return gx

In [492]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [493]:
assert y.creator
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

### 2. 역전파 도전

In [494]:
y.grad = np.array(1.0)
C = y.creator #1. 함수를 가져온다
b = C.input #2. 함수의 입력을 가져온다
b.grad = C.backward(y.grad) #3. 함수의 backward 메서드를 호출한다

In [495]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [496]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


### 3. backward 메서드 추가

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None # 변수가 어떤 함수로 부터 만들어 지는지 기록
        
    def set_creator(self, func):
        self.creator = func
        
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output #출력도 추가로 저장
        return output
        
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [499]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward() # x는 Variable 인스턴스, 자기 자신을 다시 호출하는 모양
            
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx 
        
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [500]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 8. 재귀에서 반복문으로

### 1. 현재의 Variable 클래스

* 재귀구조를 따르고 있음

### 2. 반복문을 이용한 구현

In [503]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다
            x, y = f.input, f.output # 함수의 입력과 출력을 가져온다
            x.grad = f.backward(y.grad) # backward 메서드 호출
            
            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가한다
                
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * gy * x
        return gx
                
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = gy * np.exp(x)
        return gx

In [300]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [504]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 9. 함수를 더 편리하게

### 1. 파이썬 함수로 이용하기

In [302]:
# 인스턴스를 두번 호출해야되는 불편함이 존재

x = Variable(np.array(0.5))
f = Square()
y = f(x)

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다
            x, y = f.input, f.output # 함수의 입력과 출력을 가져온다
            x.grad = f.backward(y.grad) # backward 메서드 호출
            
            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가한다
                
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * gy * x
        return gx
                
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = gy * np.exp(x)
        return gx

In [527]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            if x.creator is not None:
                funcs.append(x.creator)
                
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * gy * x
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = gy * np.exp(x)
        return gx

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [528]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [529]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) #자연스러운 함수 생성이 가능해진다
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### 2. backward 메서드 간소화

* grad : y.grad=np.array(1.0) 입력을 간소화

In [309]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) #self.data와 형상과 데이터 타입이 같은 ndarray 인스턴스를 생성, 모든 요소를 1로 채움
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [310]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


### 3. ndarray만 취급하기

In [311]:
type(y.data) # Variable입력값으로 np.array를 줬는데 np.float64로 타입이 바뀌어있음

numpy.float64

In [314]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [315]:
# type test

x = Variable(np.array(1.0))
x = Variable(None)
x = Variable(1.0)

TypeError: <class 'float'>은(는) 지원하지 않습니다.

In [316]:
# 문제점

x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

x = np.array(1.0)
y = x**2
print(type(x), x.ndim)
print(type(y)) # np.array를 연산하면 np.float으로 바뀌는걸 볼 수 있다

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [318]:
def as_array(x):
    if np.isscalar(x): # np.float은 scalar로 차원이 없다(바로 위 cell참고)
        return np.array(x)
    return x

In [319]:
import numpy as np

np.isscalar(np.float64(1.0))

True

In [320]:
print(np.isscalar(2.0))
print(np.isscalar(np.array(1.0)))
print(np.isscalar(np.array([1,2,3])))

True
False
False


In [321]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) 
        output = Variable(as_array(y)) # 함수 계산 후, 형변환이 일어나는걸 방지
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

## 10. 테스트

### 1. 파이썬 단위 테스트

In [ ]:
# step10.py 파일에서 실행
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

# 2. 자연스러운 코드로

## 1. 가변 길이 인수(순전파)

### 1. Function 클래스 수정

In [322]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    

In [323]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # 리스트형태로 여러가지 input을 받을 수 있도록 변환
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] # input이 여러개니까 output도 여러개
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()

### 2. Add 클래스 구현

In [324]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, ) # [Variable(as_array(y)) for y in ys] 여기서 ys는 iterable한 객체여야 하기때문에 tuple로 입력형태를 바꾼다

In [325]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


## 2. 가변 길이 인수(개선 편)

### 1. 첫번째 개선: 함수를 사용하기 쉽게

* f(xs) => f(x0, x1)

In [326]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0] # 지금까지는 함수 반환값이 전부 1개였음

In [327]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )

In [328]:
x0, x1 = Variable(np.array(2)), Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


### 2. 두 번째 개선: 함수를 구현하기 쉽도록

```python
# 기존함수
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)
```

```python
# 수정
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
```

In [350]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.ouputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0] 

In [351]:
# self.forward(*xs) 에서 *xs를 사용하는 이유
def t_add(x0, x1):
    y = x0 + x1
    return y

t_add(*[1,2])
#t_add([1,2]) 오류발생, 이렇게 사용하려면 기존처럼 x0, x1 = xs 형식으로 두개 인자를 받도록 해야함

3

In [352]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y # 앞에서 ys객체를 (ys, )로 return해주기때문에 여기서 (y, )형태로 반환하지 않아도 됨

참고 isinstance를 사용하여 tuple형태로 바궈주는이유
* 코드를 보면 ys변수를 반복객체로 리스트 컴프리핸션으로 사용하고 있음
* xs에 하나의 값이 들어오면 ys는 하나의 값으로 출력되고 그 다음 반복객체가 들어가야되는 리스트컴프리핸션 부분에서 오류가 발생함
* 따라서 하나의 값이 들어갔을때도 튜플로 만들어 강제로 반복객체로 만들어 오류를 피함


In [353]:
temp_var1 = (1)
temp_var2 = (1,)

print(type(temp_var1))
print(type(temp_var2))

<class 'int'>
<class 'tuple'>


In [354]:
temp_list = [var for var in temp_var2]
print(temp_list)

[1]


In [355]:
try:
    temp_list = [var for var in temp_var1]
except:
    print('error')

error


### 3. add 함수 구현

In [356]:
def add(x0, x1):
    return Add()(x0, x1)

# 위와 동일
# def add(x0, x1):
#     f = Add()
#     return f(x0, x1)

In [357]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## 3. 가변 길이 인수(역전파 편)

### 1. Add 클래스 역전파

In [358]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

### 2. Variable 클래스 수정

In [397]:
# 기존 Variable 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [398]:
# 수정 Variable 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

### 3. Square 클래스 구현

In [409]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0] 

In [410]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

In [411]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    def backward(self, gy):
        x = self.inputs[0].data # 수정 전 : x = self.input.data
        gx = 2 * x * gy
        return gx

In [412]:
def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [414]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


## 14. 같은 변수 반복 사용

같은 변수를 반복해서 사용하면 제대로 미분하지 못함

In [422]:
x = Variable(np.array(3.0))
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad) # y = x+x -> y=2x, 미분값은 2가 나와야함

y 6.0
x.grad 1.0


### 1. 문제의 원인

In [423]:
# 수정 Variable 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx # 여기서 x.grad에 미분값이 덮어 써지기때문에 오류 발생
                
                if x.creator is not None:
                    funcs.append(x.creator)

### 2. 해결책

In [424]:
# 수정 Variable 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

In [425]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad) 

y 6.0
x.grad 2.0


In [426]:
x = Variable(np.array(3.0))
y = add(add(x,x),x)
y.backward()
print(x.grad)

3.0


### 3. 미분값 재설정

* 같은 변수를 사용하여 다른 계산을 할 경우 계산이 꼬임

In [427]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

y = add(add(x,x),x)
y.backward()
print(x.grad) # x.grad의 기존값이 초기화 되지 않아 결과가 이상하게 나온다

2.0
5.0


In [428]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
    
    def cleargrad(self):
        self.grad = None

In [429]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0


## 15. 복잡한 계산 그래프 

### 1. 역전파의 올바른 순서

* 계산그래프가 복잡해질경우 역전파 때 문제가 생길수 있음

### 2. 현재의 DeZero

In [466]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

리스트의 pop과 append

* y = square(add(x0, x1))일때
* add를 이용했을때 backward상황일때
* input이 두개 따라서 마지막에 funcs.append()를 하면 [x0.creator, x1.creator]이됨
* 이때 funcs.pop()을 하기때문에 x1.creator의 역전파를 계산함
* 이후 funcs.append(x3.creator)을 함
* 그럼 funcs에는 [x0.creator, x3.creator]이되고 pop을 이용하면 x3.creator의 역전파를 계산
* 이러면 x0.creator는 마지막에 역전파가 계산됨 오류 발생

> 나중에 좀더 잘설명하길...

In [475]:
a = [1,2]
a.pop()
a.append(3)
a

[1, 3]

## 3. 함수 우선순위

* 함수에 세대를 지정한다

## 16. 복잡한 계산 그래프

### 1. 세대 추가

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

In [476]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{format(type(data))}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = None
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대 추가
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ouput.grad for ouput in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

In [477]:
class Function(object): #object는 없어도 무관 class Function(object): == class Function
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs]) #generation 추가
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]